# Classification

In [1]:
import numpy
import urllib
import scipy.optimize
import random
from sklearn import svm # support vector machine (see webpage)
from sklearn import linear_model # logistic regression
import ast

In [2]:
def parseData(fname):
    for l in urllib.urlopen(fname):
        yield eval(l)

In [3]:
def parseDataFromFile(fname):
    for l in open(fname):
        yield eval(l)

In [4]:
data1 = list(parseDataFromFile("book_descriptions_50000.json"))

In [5]:
data2 = list(parseDataFromFile("book_images_5000.json"))

In [6]:
data = data1

In [11]:
data[2]

{'asin/ID': '157120315X',
 'url': 'http://www.amazon.com/dp/157120315X',
 'description': 'quiltmaker magazine one of americas most prestigious quilting publications also created the books paper piecing picnic and paper piecing potpourri',
 'categories': ["Children's Books", 'Crafts']}

In [12]:
len(data2)

5000

In [13]:
prior = ["Children's Books" in b['categories'] for b in data]
prior = sum(prior) / len(prior)

In [14]:
prior

0.08288

In [15]:
prior_neg = 1 - prior

In [16]:
prior_neg

0.91712

p(wizard in description | Children's Book)

In [18]:
p1 = ['wizard' in b['description'] for b in data if "Children's Books" in b['categories']]
p1 = sum(p1) / len(p1)

In [19]:
p1

0.01085907335907336

In [20]:
p1_neg = ['wizard' in b['description'] for b in data if not "Children's Books" in b['categories']]
p1_neg = sum(p1_neg) / len(p1_neg)

In [21]:
p1_neg

0.004012561060711794

p(witch in description | Children's Book)

In [24]:
p2 = ['witch' in b['description'] for b in data if "Children's Books" in b['categories']]
p2 = sum(p2) / len(p2)

In [25]:
p2

0.02895752895752896

In [26]:
p2_neg = ['witch' in b['description'] for b in data if not "Children's Books" in b['categories']]
p2_neg = sum(p2_neg) / len(p2_neg)

In [27]:
p2_neg

0.018667131891137473

In [28]:
score = prior * p1 * p2
score_neg = prior_neg * p1_neg * p2_neg

In [29]:
score / score_neg

0.3793836356818883

In [30]:
p = ["Children's Books" in b['categories'] for b in data if
    'witch' in b['description'] and 'wizard' in b['description']]
p = sum(p) * 1.0 / len(p)

In [31]:
p / (1 - p)

0.4615384615384615

# Classification - Judging a book by its cover

In [32]:
data = data2

In [33]:
random.shuffle(data)

In [34]:
data[0]

{'asin/ID': '1580400299',
 'url': 'http://www.amazon.com/dp/1580400299',
 'description': 'robyn webb is the 1995 virginia recipient of the national healthy american fitness leader award by the presidents council on fitness and exercise she is also the author of a pinch of thyme easy lessons for a leaner life diabetic meals in 30 minutesor less flavorful seasons cookbook and memorable menus made easy',
 'categories': ['Cookbooks', 'Health'],
 'image_feature': [0.0,
  0.0,
  0.0,
  1.9311000108718872,
  0.0,
  0.11670000106096268,
  0.4729999899864197,
  0.0,
  0.0,
  0.08510000258684158,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.7785999774932861,
  0.0,
  1.2939000129699707,
  0.0,
  0.0,
  0.0,
  0.0,
  1.3476999998092651,
  4.499199867248535,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.4896000027656555,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.6161999702453613,
  0.0,
  0.0,
  0.0,
  0.6966999769210815,
  0.0,
 

In [35]:
X = [d['image_feature'] for d in data]
y = ["Children's Books" in d['categories'] for d in data]

In [36]:
X_train = X[:len(X)//2]
y_train = y[:len(y)//2]

X_test = X[len(X)//2:]
y_test = y[len(y)//2:]

In [37]:
mod = linear_model.LogisticRegression(C=1.0)
mod.fit(X_train, y_train)

/Users/youzezheng/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [38]:
train_predictions = mod.predict(X_train)
test_predictions = mod.predict(X_test)

In [39]:
# accuracy
correct = [a == b for a,b in zip(test_predictions, y_test)]

In [40]:
sum(correct) / len(correct)

0.9332

In [41]:
y_false = numpy.array([False for y in y_test])

In [42]:
y_false

array([False, False, False, ..., False, False, False])

In [43]:
# accuracy when predicting "false" always
sum(y_test == y_false) / len(y_test)

0.9412

In [44]:
sum(y_test)

147